## Experimentos con Gradient Boosting y Random Forest

### Diego Mercado Coello
### # de expediente: 745441

In [25]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, ParameterSampler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import mlflow
from mlflow import MlflowClient
from scipy.stats import randint, uniform
import dagshub
from dagshub import get_repo_bucket_client
from datetime import datetime
from math import sqrt
import pathlib


In [26]:
# Configurar DAGsHub como el tracking server de MLflow
dagshub.init(url="https://dagshub.com/diego-mercadoc/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()
print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Initialized MLflow to track repo "diego-mercadoc/nyc-taxi-time-prediction"

Repository diego-mercadoc/nyc-taxi-time-prediction initialized!

https://dagshub.com/diego-mercadoc/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/4a2f89adc0ff477ead9bc8f38ff1a8ba', creation_time=1726630128698, experiment_id='0', last_update_time=1726630128698, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [27]:
# Definir función para cargar los datos
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [28]:
# Cargar los datos de entrenamiento y validación
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [29]:
# Definir función para preprocesar los datos (Feature Engineering)
def preprocess_data(df_train, df_val):
    df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
    df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dv = DictVectorizer()

    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dicts)

    y_train = df_train['duration'].values
    y_val = df_val['duration'].values

    return X_train, X_val, y_train, y_val, dv

In [30]:
# Preprocesar los datos
X_train, X_val, y_train, y_val, dv = preprocess_data(df_train, df_val)

In [31]:
# Subir los datasets a DAGsHub Storage
s3 = get_repo_bucket_client("diego-mercadoc/nyc-taxi-time-prediction")

# Upload training data
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",
    Filename="../data/green_tripdata_2024-01.parquet",
    Key="train_data.parquet",
)

# Upload validation data
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",
    Filename="../data/green_tripdata_2024-02.parquet",
    Key="eval_data.parquet",
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [32]:
# Definir función para entrenar Gradient Boosting con runs anidados
def train_gradient_boosting(X_train, y_train, X_val, y_val, dv):
    from sklearn.model_selection import ParameterSampler
    gb_params = {
        'n_estimators': randint(50, 200),
        'learning_rate': uniform(0.01, 0.3),
        'max_depth': randint(3, 10)
    }
    param_distributions = list(ParameterSampler(gb_params, n_iter=10, random_state=42))
    best_rmse = float('inf')
    best_model = None
    best_params = None
    with mlflow.start_run(run_name="GradientBoostingRegressor") as parent_run:
        mlflow.set_tag("model_family", "GradientBoostingRegressor")
        for params in param_distributions:
            with mlflow.start_run(nested=True):
                mlflow.log_params(params)
                model = GradientBoostingRegressor(**params)
                model.fit(X_train, y_train)
                y_pred = model.predict(X_val)
                rmse = sqrt(mean_squared_error(y_val, y_pred, squared=False))
                mlflow.log_metric("rmse", rmse)
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = model
                    best_params = params
                    best_run_id = mlflow.active_run().info.run_id
        # Loguear el mejor modelo en el run padre
        mlflow.log_params(best_params)
        mlflow.log_metric("best_rmse", best_rmse)
        mlflow.sklearn.log_model(best_model, artifact_path="model")
        # Log preprocessor
        pathlib.Path("models").mkdir(exist_ok=True)
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    return best_rmse, parent_run.info.run_id

### Definir función para entrenar Random Forest con runs anidados

In [33]:
def train_random_forest(X_train, y_train, X_val, y_val, dv):
    from sklearn.model_selection import ParameterSampler
    rf_params = {
        'n_estimators': randint(50, 200),
        'max_depth': randint(3, 20),
        'max_features': ['sqrt', 'log2', None]
    }
    param_distributions = list(ParameterSampler(rf_params, n_iter=10, random_state=42))
    best_rmse = float('inf')
    best_model = None
    best_params = None
    with mlflow.start_run(run_name="RandomForestRegressor") as parent_run:
        mlflow.set_tag("model_family", "RandomForestRegressor")
        for params in param_distributions:
            with mlflow.start_run(nested=True):
                mlflow.log_params(params)
                model = RandomForestRegressor(**params)
                model.fit(X_train, y_train)
                y_pred = model.predict(X_val)
                rmse = sqrt(mean_squared_error(y_val, y_pred, squared=False))
                mlflow.log_metric("rmse", rmse)
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = model
                    best_params = params
                    best_run_id = mlflow.active_run().info.run_id
        # Loguear el mejor modelo en el run padre
        mlflow.log_params(best_params)
        mlflow.log_metric("best_rmse", best_rmse)
        mlflow.sklearn.log_model(best_model, artifact_path="model")
        # Log preprocessor
        pathlib.Path("models").mkdir(exist_ok=True)
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    return best_rmse, parent_run.info.run_id

In [34]:
# Entrenar los modelos
rmse_gbr, run_id_gbr = train_gradient_boosting(X_train, y_train, X_val, y_val, dv)
rmse_rfr, run_id_rfr = train_random_forest(X_train, y_train, X_val, y_val, dv)

c:\Users\diego\OneDrive - ITESO\ITESO\5 semestre\Proyecto de Ciencia de datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/09/23 21:31:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run nervous-hawk-938 at: https://dagshub.com/diego-mercadoc/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/f0127ba7743247c1a128278cd2aaa240.
2024/09/23 21:31:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/diego-mercadoc/nyc-taxi-time-prediction.mlflow/#/experiments/0.
c:\Users\diego\OneDrive - ITESO\ITESO\5 semestre\Proyecto de Ciencia de datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate t

In [35]:
# Imprimir los RMSE obtenidos
print(f"RMSE GradientBoostingRegressor: {rmse_gbr}")
print(f"RMSE RandomForestRegressor: {rmse_rfr}")

RMSE GradientBoostingRegressor: 2.310325827593497
RMSE RandomForestRegressor: 2.329380261101746


In [36]:
# Determinar el mejor modelo
if rmse_gbr < rmse_rfr:
    best_rmse = rmse_gbr
    best_model_name = "GradientBoostingRegressor"
    best_run_id = run_id_gbr
else:
    best_rmse = rmse_rfr
    best_model_name = "RandomForestRegressor"
    best_run_id = run_id_rfr

print(f"Mejor modelo: {best_model_name} con RMSE: {best_rmse}")

Mejor modelo: GradientBoostingRegressor con RMSE: 2.310325827593497


### Registrar el mejor modelo en MLflow y asignar el alias 'challenger'

In [37]:
# Registrar el nuevo modelo
result = mlflow.register_model(
    model_uri=f"runs:/{best_run_id}/model",
    name="nyc-taxi-model"
)

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Modelo para predecir la duración de viajes en taxi en Nueva York"
)

new_version = result.version

# Asignar el alias 'challenger' al nuevo modelo
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias="challenger",
    version=new_version
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/09/23 21:33:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 5
Created version '5' of model 'nyc-taxi-model'.


#### Descargar el conjunto de datos de marzo de 2024

In [38]:
# Descargar los datos de marzo de 2024
!curl -o ../data/green_tripdata_2024-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1340k  100 1340k    0     0  5247k      0 --:--:-- --:--:-- --:--:-- 5297k


### Cargar los modelos 'champion' y 'challenger'

In [39]:
champion_model = mlflow.pyfunc.load_model(model_uri="models:/nyc-taxi-model@champion")
challenger_model = mlflow.pyfunc.load_model(model_uri=f"models:/nyc-taxi-model/{new_version}")

2024/09/23 21:33:34 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.5.1, required: scikit-learn==1.5.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


### Evaluar ambos modelos en el conjunto de prueba

In [40]:
# Leer el conjunto de prueba
df_test = read_dataframe('../data/green_tripdata_2024-03.parquet')

In [41]:
# Preparar los datos de prueba
def prepare_test_data(df_test, dv):
    df_test['PU_DO'] = df_test['PULocationID'] + '_' + df_test['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    test_dicts = df_test[categorical + numerical].to_dict(orient='records')
    X_test = dv.transform(test_dicts)
    y_test = df_test['duration'].values

    return X_test, y_test

In [42]:
# Obtener X_test y y_test
X_test, y_test = prepare_test_data(df_test, dv)

In [43]:
# Evaluar el modelo champion
start_time = datetime.now()

y_pred_champion = champion_model.predict(X_test)
rmse_champion = sqrt(mean_squared_error(y_test, y_pred_champion, squared=False))
end_time = datetime.now()

champion_time = end_time - start_time

print(f"Tiempo de ejecución del modelo champion: {champion_time}")
print(f"RMSE Champion: {rmse_champion}")

Tiempo de ejecución del modelo champion: 0:00:00.168165
RMSE Champion: 2.2814229402623813


c:\Users\diego\OneDrive - ITESO\ITESO\5 semestre\Proyecto de Ciencia de datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [44]:
# Evaluar el modelo challenger
start_time = datetime.now()

y_pred_challenger = challenger_model.predict(X_test)
rmse_challenger = sqrt(mean_squared_error(y_test, y_pred_challenger, squared=False))
end_time = datetime.now()

challenger_time = end_time - start_time

print(f"Tiempo de ejecución del modelo challenger: {challenger_time}")
print(f"RMSE Challenger: {rmse_challenger}")

Tiempo de ejecución del modelo challenger: 0:00:00.160416
RMSE Challenger: 2.317518332546272


c:\Users\diego\OneDrive - ITESO\ITESO\5 semestre\Proyecto de Ciencia de datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Decidir si promover el modelo 'challenger' a 'champion'

In [45]:
if rmse_challenger < rmse_champion:
    # Promover challenger a champion
    client.set_registered_model_alias(
        name="nyc-taxi-model",
        alias="champion",
        version=new_version
    )
    print("El modelo 'challenger' ha sido promovido a 'champion'.")
else:
    print("El modelo 'challenger' no superó al 'champion' y no será promovido.")

El modelo 'challenger' no superó al 'champion' y no será promovido.


# Análisis para Decidir la Promoción del Modelo Challenger 
### `(hice varios experimentos, por lo que mi champion es diferente al de la clase, por eso no promoví el challenger a champion)`

### Resultados Obtenidos:

- **RMSE del Modelo Champion**: **2.2814**
- **RMSE del Modelo Challenger**: **2.3175**
- **Tiempo de Predicción/inferencia del Modelo Champion**: **0:00:00.168165 segundos**
- **Tiempo de Predicción/inferencia del Modelo Challenger**: **0:00:00.160416 segundos**

---

### 1. Rendimiento (RMSE):

- El **modelo champion** tiene un RMSE **menor** que el **modelo challenger**, que es una mejor precisión en las predicciones.
- La diferencia en RMSE es de **0.0361**, lo que creeeo que puede ser significarivo pero pues depende de contexto, me imagino, tendra que saber mas del contexto.

**Conclusión:** En cuanto a de precisión, el **modelo champion** es mejor wuuuu.

---

### 2. Tiempo de Inferencia/entrenamiento:

- El **modelo challenger** es **0.041 segundos** más rápido que el **modelo champion**.
- Esto es una mejora de alrededor del **20%** en el tiempo.

**Análisis:**

- Si el sistema necesita predicciones en tiempo real y tiene muchas solicitudes, esta mejora en tiempo podría ser importante creo.
- Si el tiempo de respuesta no es muuy importante, la mejora en velocidad puede no justificar una menor precisión.

**Conclusión:** Aunque el **modelo challenger** es más rápido, la diferencia en tiempo es mínima y debe verse y evaluar si es muy importaante para los objetivos del contexto de los taxiss.

---

### 3. Complejidad del Modelo y Uso de Recursos:

- **Modelo Champion:** Podría ser más complejo o tener hiperparámetros que aumentan su capacidad predictiva.
- **Modelo Challenger:** Puede ser más sencillo o estar optimizado para velocidad.

**Análisis:**

- Un modelo menos complejo es más fácil de mantener y deployearr.
- Si la complejidad extra del **modelo champion** no afecta muucho a los recursos, este factor no es comoo muy determinante o important.

**Conclusión:** Si los recursos y la complejidad no son limitantes, el **modelo champion** sigue siendo preferible por su mayor precisión.

---

### 4. Hiperparámetros y Generalización:

- **Modelo Challenger:** Revisar si los hiperparámetros seleccionados permiten una mejor generalización.
- **Modelo Champion:** Asegurarse de que no esté overfitteado al los datos de train.

**Análisis:**

- Evaluar otras métricas como MAE (Error Absoluto Medio).
- Realizar cross validation adicional para verificar la estabilidad del modelo.

**Conclusión:** No hay evidencia de overfitteao en el **modelo champion**; su mejor RMSE lo mantiene como la mejor opción.

---

### 5. Impacto en el supuesto negocio/empresa y Requerimientos:

- **Precisión vs. Velocidad:** Determinar si el eonteexto del negociuo valora más la precisión o la velocidad de respuesta.
- **Experiencia del Usuario:** Una mejora en el tiempo de respuesta mejora significativamente la experiencia del usuario.
- **Costos y Riesgos:** Cambiar el modelo en producción implica costos y riesgos asociados.

**Conclusión:** Si la precisión es más importants y la mejora en velocidad no tiene un impacto muy importante/relevante, es preferible mantener el **modelo champion**.

---

### **Decisión Final:**

**No se recomienda promover el modelo challenger a champion en este momento.**

---

### **Justificación:**

- **Mayor Precisión del Champion:** Ofrece predicciones más exactas, que para la satisfacción del cliente (QUE EL CLIENTE ES LA GAS DEL NEGOCIO) y confiabilidad del servicio.
- **Diferencia de Tiempo Mínima:** La mejora en tiempo de inferencia del challenger es mínima y no compensa la pérdida de precisión.
- **Riesgos Asociados:** Cambiar el modelo podría introducir riesgos sin beneficios que valgan la oena.

---

### **Acciones como a futuro:**

- **Optimización Continua:** ver otras técnicas o ajustar hiperparámetros para mejorar futuros modelos challengers.
- **Monitoreo Constante:** evr el desempeño de ambos modelos, ya que cambios en los datos podrían alterar su rendimiento.
- **Consulta con el Negocio:** Confirmar si los requerimientos podrían cambiar, haciendo que la velocidad sea más importante.

---

**POSDATA PROFE:** La decisión se basa en un análisis de los resultados y pues falta saber mas sobre los objetivos del negocio/contexto.
